In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

In [2]:
# read comments cvs
# df_comments = pd.read_csv("Manual checking - comments.csv")
df_comments = pd.read_csv("./data/pipeline/cq_parent_feats.csv")
df_comments

,Id,PostId,Score,Text,CreationDate,UserId,ContentLicense,of_answer,cnt_keywords,subject,...,ViewCount,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,ClosedDate,FavoriteCount,CommunityOwnedDate
0,1027927,1204191,0,Do you have a contact over at Amazon? They nee...,2009-07-30T05:38:18.620,76337,CC BY-SA 2.5,False,1,NaN,...,10400.0,2009-08-03T03:09:04.357,2009-09-05T04:11:20.040,How to sign an Amazon web service request in ....,<c#><wcf><web-services><soap><amazon-web-servi...,4.0,2.0,NaN,3.0,NaN
1,2301473,1496066,4,This question and answers are out of date now ...,2010-02-25T02:48:51.043,50277,CC BY-SA 2.5,False,1,question,...,4772.0,2013-01-06T13:29:42.057,2018-11-17T00:09:24.550,Amazon SimpleDB Woes: Implementing counter att...,<amazon-web-services><amazon-simpledb>,6.0,1.0,NaN,7.0,NaN
2,2301480,327169,1,This question and answers are out of date now ...,2010-02-25T02:51:10.927,50277,CC BY-SA 2.5,False,1,question,...,959.0,NaN,2008-11-29T13:39:08.123,"How far can you really go with ""eventual"" cons...",<sql><database><amazon-web-services><amazon-si...,2.0,1.0,NaN,4.0,NaN
3,8008147,6735850,0,"@house9, I've looked at heroku already, and th...",2011-07-19T22:43:03.567,152092,CC BY-SA 3.0,False,1,I,...,2907.0,NaN,2013-03-19T11:17:48.443,Rails 3 Server on EC2,<ruby-on-rails-3><amazon-ec2><ruby-on-rails-3.1>,3.0,3.0,NaN,3.0,NaN
4,9594466,7864988,0,UIDevice `uniqueIdentifier` is deprecated in i...,2011-10-23T08:13:47.107,18091,CC BY-SA 3.0,False,1,UIDevice,...,783.0,NaN,2011-10-23T09:05:42.187,ios 5 AWS SDK S3PutObjectRequest,<amazon-s3><amazon-web-services><ios5>,1.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,111881787,63241324,0,@BenT We are thinking of using timestamps to i...,2020-08-05T17:36:12.297,2184946,CC BY-SA 4.0,False,1,NaN,...,21.0,NaN,2020-08-11T20:48:26.050,How to avoid retrieving outdated data from AWS...,<amazon-web-services><iot><aws-iot>,1.0,2.0,NaN,NaN,NaN
367,112537429,63640052,0,the only error thats in there is this warning ...,2020-08-28T20:32:43.217,14184500,CC BY-SA 4.0,False,1,Buffer,...,34.0,2020-08-28T22:39:27.823,2020-08-28T22:39:27.823,S3.upload not working in production environmen...,<node.js><amazon-web-services><amazon-s3><netl...,0.0,10.0,NaN,NaN,NaN
368,112577969,63644158,0,"Yes, using a load balancer would be ideal, but...",2020-08-31T01:47:22.793,12564768,CC BY-SA 4.0,False,1,NaN,...,19.0,2020-09-02T01:46:24.570,2020-09-02T01:46:24.570,ECS randomly terminates instances and starts a...,<amazon-ec2><amazon-ecs>,0.0,3.0,NaN,NaN,NaN
369,112606415,63679395,2,Do you absolutely need a 0.11 compatible answe...,2020-08-31T23:46:36.197,2291321,CC BY-SA 4.0,False,1,0.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_answers = pd.read_csv("./data/pipeline/ca_parent_feats.csv")
# df_answers.columns
df_answers = df_answers.drop('Body',axis = 1)
# df_answers['Id_ans']==df_answers['Id']

In [5]:
df_comments.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId',
       'ContentLicense', 'of_answer', 'cnt_keywords', 'subject', 'punctuation',
       'negative_statement', 'sentence', 'subj_irrel', 'include_irrel',
       'Id_que', 'CreationDate_que', 'Score_que', 'ViewCount', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'ClosedDate', 'FavoriteCount', 'CommunityOwnedDate'],
      dtype='object')

In [6]:
set(df_comments.columns)-set(df_answers.columns)

{'CommentCount', 'LastActivityDate'}

In [7]:
set(df_answers.columns)-set(df_comments.columns)

{'CommentCount_ans',
 'CommentCount_que',
 'CreationDate_ans',
 'Id_ans',
 'LastActivityDate_ans',
 'LastActivityDate_que',
 'ParentId',
 'Score_ans'}

In [8]:
print(len(df_comments.columns))
print(len(df_answers.columns))

28
34


In [9]:
df = pd.concat([df_comments,df_answers],axis = 0,ignore_index = True)

/Users/jianghuimin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 36 columns):
AnswerCount             697 non-null float64
ClosedDate              21 non-null object
CommentCount            355 non-null float64
CommentCount_ans        342 non-null object
CommentCount_que        342 non-null float64
CommunityOwnedDate      0 non-null float64
ContentLicense          948 non-null object
CreationDate            948 non-null object
CreationDate_ans        342 non-null object
CreationDate_que        697 non-null object
FavoriteCount           249 non-null float64
Id                      948 non-null int64
Id_ans                  342 non-null float64
Id_que                  697 non-null float64
LastActivityDate        355 non-null object
LastActivityDate_ans    342 non-null float64
LastActivityDate_que    342 non-null object
LastEditDate            364 non-null object
ParentId                342 non-null float64
PostId                  948 non-null int64
Score      

In [11]:
#CommunityOwnedDate      0 non-null float64
#drop the columns completely lost
df = df.drop('CommunityOwnedDate',axis = 1)

In [12]:
def Find(string):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)
    url_list=[x[0] for x in url]
    if len(url_list) != 0:
        return 1
    else:
        return 0
df['share_link']= df['Text'].apply(lambda x:Find(x))

In [14]:
def get_tags(tags):
#     print(tags)
    if tags != tags:
        return None
    else:
        tags = tags[1:-1]
        tags = tags.split('><')
        res = []
        for item in tags:
            if ('aws' in item) or ('amazon' in item):
                if item != 'aws' and item !='amazon-web-services':
                    res.append(item)
        if res == []:
            return None
        return res
df['aws_related_tags']= df['Tags'].apply(lambda x:get_tags(x))

In [15]:
def count_tag_num(tags):
    if tags == None:
        return None
    return len(tags)
df['tag_num']= df['aws_related_tags'].apply(lambda x:count_tag_num(x))

In [16]:
pd.unique(df['tag_num'])

array([nan,  1.,  2.,  3.,  4.])

In [17]:
def get_tags(tags):
    if tags != tags:
        return None
    else:
        tags = tags[1:-1]
        tags = tags.split('><')
        res = []
        for item in tags:
            if ('aws' in item) or ('amazon' in item):
                if item != 'aws' and item !='amazon-web-services':
                    res.append(item)
        if res == []:
            return None
        return res
df['aws_related_tags']= df['Tags'].apply(lambda x:get_tags(x))

In [18]:
def get_code(text):
    code = re.findall('`([^"]*)`',text)
    if code ==[]:
        return 0
    else:
        return 1
df['share_code']= df['Text'].apply(lambda x:get_code(x))

In [19]:
# read answers cvs
# df_answers = pd.read_csv("Manual checking - answers.csv")
df_answers_checked = pd.read_csv("./data/pipeline/Manual checking - answers (1).csv")
df_answers_checked = df_answers_checked[:100]

In [20]:
df_answers_checked = df_answers_checked[['answer_id','outdated (manually checked)']]

In [21]:
df = df.merge(df_answers_checked,left_on='Id_ans',right_on='answer_id',how = 'left')

In [22]:
# read comments cvs
# df_comments = pd.read_csv("Manual checking - comments.csv")
df_comments_checked = pd.read_csv("./data/pipeline/Manual checking - comments(new).csv")
df_comments_checked = df_comments_checked[:194]


In [23]:
# df_comments_checked 

In [24]:
df_comments_checked = df_comments_checked[['Id','outdated (manually checked)']]

In [25]:
df = df.merge(df_comments_checked,left_on='Id',right_on='Id',how = 'left')

In [26]:
def get_outdated(x,y):
    if x == x:
        return x
    elif y == y:
        return y
    else:
        return None
df['outdated (manually checked)']= df.apply(lambda x:get_outdated(x['outdated (manually checked)_x'],x['outdated (manually checked)_y']),axis =1)

In [27]:
def train_or_test(manual):
    if manual == None:
        return 'Test'
    else:
        return 'Train'
df['is_training']= df['outdated (manually checked)'].apply(lambda x:train_or_test(x))
 

In [28]:
df = df.drop(['outdated (manually checked)_x','outdated (manually checked)_y','answer_id'],axis = 1)

In [29]:
df = df.drop(['tag_num'],axis =1)

In [33]:
def get_keywords(text):
    text_dict = ['outdated','deprecated','obsolete','out of date','not deprecated','old version','probably outdated','remove','instead']
    res = []
    for item in text_dict:
        if item in text:
            res.append(item)
    res = set(res)
    if 'not deprecated' in res and 'deprecated' in res:
        res.remove('deprecated')
    return res
df['keywords']= df['Text'].apply(lambda x:get_keywords(x))

In [34]:
df

,AnswerCount,ClosedDate,CommentCount,CommentCount_ans,CommentCount_que,ContentLicense,CreationDate,CreationDate_ans,CreationDate_que,FavoriteCount,...,punctuation,sentence,subj_irrel,subject,share_link,aws_related_tags,share_code,outdated (manually checked),is_training,keywords
0,4.0,NaN,2.0,NaN,NaN,CC BY-SA 2.5,2009-07-30T05:38:18.620,NaN,2009-07-30T02:48:06.287,3.0,...,.,They need to be made to understand why they sh...,False,NaN,0,None,0,None,Test,{obsolete}
1,6.0,NaN,1.0,NaN,NaN,CC BY-SA 2.5,2010-02-25T02:48:51.043,NaN,2009-09-30T04:04:17.807,7.0,...,.,This question and answers are outdated now tha...,False,question,1,[amazon-simpledb],0,None,Test,{out of date}
2,2.0,NaN,1.0,NaN,NaN,CC BY-SA 2.5,2010-02-25T02:51:10.927,NaN,2008-11-29T04:36:13.523,4.0,...,.,This question and answers are outdated now tha...,False,question,1,[amazon-simpledb],0,True,Train,{out of date}
3,3.0,NaN,3.0,NaN,NaN,CC BY-SA 3.0,2011-07-19T22:43:03.567,NaN,2011-07-18T16:06:30.553,3.0,...,.,"Regarding the other link, I've looked them up,...",False,I,0,[amazon-ec2],0,None,Test,{outdated}
4,1.0,NaN,2.0,NaN,NaN,CC BY-SA 3.0,2011-10-23T08:13:47.107,NaN,2011-10-23T07:47:23.067,NaN,...,.,UIDevice `uniqueIdentifier` is deprecated in i...,False,UIDevice,0,[amazon-s3],1,None,Test,{deprecated}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,1.0,NaN,NaN,CC BY-SA 4.0,0.0,CC BY-SA 4.0,2020-08-06T06:06:30.027,2020-08-06T00:43:56.577,2020-08-06T00:31:46.233,NaN,...,,"and there are many different solutions, but th...",False,NaN,0,"[amazon-ec2, amazon-elastic-beanstalk]",0,False,Train,{outdated}
948,NaN,NaN,NaN,NaN,NaN,CC BY-SA 4.0,2020-08-06T11:44:24.747,NaN,NaN,NaN,...,.,"Nugets were pretty outdated, scrub move on my ...",False,Nugets,0,None,0,None,Test,{out of date}
949,1.0,NaN,NaN,CC BY-SA 4.0,0.0,CC BY-SA 4.0,2020-08-25T15:29:15.263,2020-08-24T14:09:48.990,2020-08-24T08:40:14.813,NaN,...,.,missing AWS credentials not configured using A...,False,NaN,0,None,0,None,Test,{outdated}
950,1.0,NaN,NaN,CC BY-SA 4.0,1.0,CC BY-SA 4.0,2020-08-31T05:31:07.820,2020-08-31T00:19:50.087,2020-01-03T10:28:24.363,NaN,...,,AWS make changes to the process and it would l...,False,NaN,0,[aws-glue],0,None,Test,{out of date}


In [36]:
df.to_csv("./data/pipeline/952_rows_41_columns.csv")